# 知识图谱的制作

In [1]:
import os
import json
from py2neo import Graph,Node

In [2]:
data_path = 'data/medical2.json' # 获取json文件路径
g = Graph("neo4j://localhost:7687", auth=("neo4j", "-"))# 连接数据库

In [3]:
def read_nodes():
    # 共7类节点，节点的选择与业务实际相关

    checks = [] # 检查
    departments = [] # 科室
    diseases = [] # 疾病
    drugs = [] # 药品
    symptoms = [] # 症状
    producers = [] # 药品制造商
    foods = [] # 食物
    
    #这一项没有出现在节点中，在后面编程中用于创建疾病信息表
    disease_infos = []#疾病信息

    # 构建节点实体关系,共11类，medical2做出来的只有10类，因为数据量少
    rels_department = [] #　科室－科室关系，这一项关系PPT中未列出，后加入
    rels_noteat = [] # 疾病－忌吃食物关系
    rels_doeat = [] # 疾病－宜吃食物关系
    rels_recommandeat = [] # 疾病－推荐吃食物关系
    rels_commonddrug = [] # 疾病－通用药品关系
    rels_recommanddrug = [] # 疾病－热门药品关系
    rels_check = [] # 疾病－检查关系
    rels_drug_producer = [] # 厂商－药物关系
    rels_symptom = [] #疾病症状关系
    rels_acompany = [] # 疾病并发关系
    rels_category = [] #　疾病与科室之间的关系

    count = 0
    for data in open(data_path):
        disease_dict = {}
        count += 1
        print(count)
        data_json = json.loads(data)#读取数据
        disease = data_json['name']
        disease_dict['name'] = disease
        diseases.append(disease)
        disease_dict['desc'] = ''
        disease_dict['prevent'] = ''
        disease_dict['cause'] = ''
        disease_dict['easy_get'] = ''
        disease_dict['cure_department'] = ''
        disease_dict['cure_way'] = ''
        disease_dict['cure_lasttime'] = ''
        disease_dict['symptom'] = ''
        disease_dict['cured_prob'] = ''

        #查找一下词条是否在提取出来的文档段中，每一条文档段内容长度不一
        if 'symptom' in data_json:
            symptoms += data_json['symptom']# +号用于组合列表
            for symptom in data_json['symptom']:#用for循环的原因：一个疾病可能对应好几个症状，手画图表示
                rels_symptom.append([disease, symptom]) # 对于每个症状都建立一个疾病——症状的关系

        if 'acompany' in data_json:
            for acompany in data_json['acompany']:#同上，一个疾病可能伴随多个并发症
                rels_acompany.append([disease, acompany])# 建立一个疾病——伴随疾病的关系

        if 'desc' in data_json:
            disease_dict['desc'] = data_json['desc']#疾病描述，这里不是关系，而是定义的属性。根据业务要求把一个字段定义为关系或者节点的属性

        if 'prevent' in data_json:
            disease_dict['prevent'] = data_json['prevent']#疾病预防

        if 'cause' in data_json:
            disease_dict['cause'] = data_json['cause']#引起疾病的原因

        if 'get_prob' in data_json:
            disease_dict['get_prob'] = data_json['get_prob']#发病率

        if 'easy_get' in data_json:
            disease_dict['easy_get'] = data_json['easy_get']#易感人群

        if 'cure_department' in data_json:
            cure_department = data_json['cure_department']#治疗科室
            if len(cure_department) == 1:
                rels_category.append([disease, cure_department[0]])#只有一个科室的情况
            if len(cure_department) == 2:
                big = cure_department[0]#大科室
                small = cure_department[1]#细分小科室
                rels_department.append([small, big])  # 提取科室——科室关系
                rels_category.append([disease, small])#提取疾病——科室关系

            disease_dict['cure_department'] = cure_department
            departments += cure_department

        if 'cure_way' in data_json:
            disease_dict['cure_way'] = data_json['cure_way']#治疗途径

        if 'cure_lasttime' in data_json:
            disease_dict['cure_lasttime'] = data_json['cure_lasttime']#治疗时间

        if 'cured_prob' in data_json:
            disease_dict['cured_prob'] = data_json['cured_prob']#治愈概率

        if 'common_drug' in data_json:
            common_drug = data_json['common_drug']#常用药物
            for drug in common_drug:
                rels_commonddrug.append([disease, drug])#提取疾病——药物的关系
            drugs += common_drug

        if 'recommand_drug' in data_json:
            recommand_drug = data_json['recommand_drug']#推荐药物
            drugs += recommand_drug
            for drug in recommand_drug:
                rels_recommanddrug.append([disease, drug])#提取疾病——推荐药物的关系

        if 'not_eat' in data_json:
            not_eat = data_json['not_eat']#不能吃的食物
            for _not in not_eat:
                rels_noteat.append([disease, _not])#提取疾病——不能吃的食物的关系

            foods += not_eat
            do_eat = data_json['do_eat']#可以吃的食物
            for _do in do_eat:
                rels_doeat.append([disease, _do])#提取疾病——能吃的食物的关系

            foods += do_eat
            recommand_eat = data_json['recommand_eat']#推荐吃的食物

            for _recommand in recommand_eat:
                rels_recommandeat.append([disease, _recommand])#提取疾病——推荐吃的食物的关系
            foods += recommand_eat

        if 'check' in data_json:
            check = data_json['check']#检查项，一个疾病对应多个检查
            for _check in check:
                rels_check.append([disease, _check])#提取疾病——检查项的关系
            checks += check
        if 'drug_detail' in data_json:
            drug_detail = data_json['drug_detail']#药物详细信息
            producer = [i.split('(')[0] for i in drug_detail]
            rels_drug_producer += [[i.split('(')[0], i.split('(')[-1].replace(')', '')] for i in drug_detail]
            producers += producer
        disease_infos.append(disease_dict)#添加疾病信息list
    return set(drugs), set(foods), set(checks), set(departments), set(producers), set(symptoms), set(diseases), disease_infos,\
            rels_check, rels_recommandeat, rels_noteat, rels_doeat, rels_department, rels_commonddrug, rels_drug_producer, rels_recommanddrug,\
            rels_symptom, rels_acompany, rels_category

In [4]:
'''建立节点'''
def create_node(label, nodes):
    count = 0
    for node_name in nodes:
        node = Node(label, name=node_name)
        g.create(node)
        count += 1
        print(count, len(nodes))
    return

In [5]:
'''创建知识图谱中心疾病的节点'''
def create_diseases_nodes(disease_infos):
    count = 0
    for disease_dict in disease_infos:
        node = Node("Disease", name=disease_dict['name'], desc=disease_dict['desc'],
                    prevent=disease_dict['prevent'], cause=disease_dict['cause'],
                    easy_get=disease_dict['easy_get'], cure_lasttime=disease_dict['cure_lasttime'],
                    cure_department=disease_dict['cure_department']
                    ,cure_way=disease_dict['cure_way'], cured_prob=disease_dict['cured_prob'])#各个疾病节点的属性
        g.create(node)
        count += 1
        print(count)
    return

In [6]:
'''创建知识图谱实体节点类型schema,节点个数多，创建过程慢'''
def create_graphnodes():
    Drugs, Foods, Checks, Departments, Producers, Symptoms, Diseases, disease_infos,rels_check, rels_recommandeat, rels_noteat, rels_doeat, rels_department, rels_commonddrug, rels_drug_producer, rels_recommanddrug,rels_symptom, rels_acompany, rels_category = read_nodes()
    create_diseases_nodes(disease_infos)#调用上面的疾病节点创建函数
    create_node('Drug', Drugs)#创建药物节点
    print(len(Drugs))
    create_node('Food', Foods)#创建食物节点
    print(len(Foods))
    create_node('Check', Checks)#创建检查节点
    print(len(Checks))
    create_node('Department', Departments)#创建科室节点
    print(len(Departments))
    create_node('Producer', Producers)#创建制药厂节点
    print(len(Producers))
    create_node('Symptom', Symptoms)#创建症状节点
    return

In [7]:
'''创建实体关联边'''
def create_relationship(start_node, end_node, edges, rel_type, rel_name):#起点节点，终点节点，边，关系类型，关系名字
    count = 0
    # 去重处理
    set_edges = []
    for edge in edges:
        set_edges.append('###'.join(edge))#使用###作为不同关系之间分隔的标志
    all = len(set(set_edges))
    for edge in set(set_edges):
        edge = edge.split('###')#选取前两个关系，因为两个节点之间一般最多两个关系
        p = edge[0]
        q = edge[1]
        query = "match(p:%s),(q:%s) where p.name='%s'and q.name='%s' create (p)-[rel:%s{name:'%s'}]->(q)" % (
            start_node, end_node, p, q, rel_type, rel_name)#match语法，p，q分别为标签，rel_type关系类别，rel_name 关系名字
        try:
            g.run(query)#执行语句
            count += 1
            print(rel_type, count, all)
        except Exception as e:
            print(e)
    return

In [8]:
'''创建11种实体关系边'''
def create_graphrels():
    Drugs, Foods, Checks, Departments, Producers, Symptoms, Diseases, disease_infos, rels_check, rels_recommandeat, rels_noteat, rels_doeat, rels_department, rels_commonddrug, rels_drug_producer, rels_recommanddrug,rels_symptom, rels_acompany, rels_category = read_nodes()
    create_relationship('Disease', 'Food', rels_recommandeat, 'recommand_eat', '推荐食谱')#调用下面的关系边创建函数
    create_relationship('Disease', 'Food', rels_noteat, 'no_eat', '忌吃')
    create_relationship('Disease', 'Food', rels_doeat, 'do_eat', '宜吃')
    create_relationship('Department', 'Department', rels_department, 'belongs_to', '属于')
    create_relationship('Disease', 'Drug', rels_commonddrug, 'common_drug', '常用药品')
    create_relationship('Producer', 'Drug', rels_drug_producer, 'drugs_of', '生产药品')
    create_relationship('Disease', 'Drug', rels_recommanddrug, 'recommand_drug', '好评药品')
    create_relationship('Disease', 'Check', rels_check, 'need_check', '诊断检查')
    create_relationship('Disease', 'Symptom', rels_symptom, 'has_symptom', '症状')
    create_relationship('Disease', 'Disease', rels_acompany, 'acompany_with', '并发症')
    create_relationship('Disease', 'Department', rels_category, 'belongs_to', '所属科室')


In [14]:
'''导出数据'''
def export_data():
    Drugs, Foods, Checks, Departments, Producers, Symptoms, Diseases, disease_infos, rels_check, rels_recommandeat, rels_noteat, rels_doeat, rels_department, rels_commonddrug, rels_drug_producer, rels_recommanddrug, rels_symptom, rels_acompany, rels_category = read_nodes()
    f_drug = open('./dict/drug.txt', 'w+')
    f_food = open('./dict/food.txt', 'w+')
    f_check = open('./dict/check.txt', 'w+')
    f_department = open('./dict/department.txt', 'w+')
    f_producer = open('./dict/producer.txt', 'w+')
    f_symptom = open('./dict/symptom.txt', 'w+')
    f_disease = open('./dict/disease.txt', 'w+')

    f_drug.write('\n'.join(list(Drugs)))
    f_food.write('\n'.join(list(Foods)))
    f_check.write('\n'.join(list(Checks)))
    f_department.write('\n'.join(list(Departments)))
    f_producer.write('\n'.join(list(Producers)))
    f_symptom.write('\n'.join(list(Symptoms)))
    f_disease.write('\n'.join(list(Diseases)))

    f_drug.close()
    f_food.close()
    f_check.close()
    f_department.close()
    f_producer.close()
    f_symptom.close()
    f_disease.close()

    return

In [10]:
create_graphnodes()#创建节点

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
1 65
2 65
3 65
4 65
5 65
6 65
7 65
8 65
9 65
10 65
11 65
12 65
13 65
14 65
15 65
16 65
17 65
18 65
19 65
20 65
21 65
22 65
23 65
24 65
25 65
26 65
27 65
28 65
29 65
30 65
31 65
32 65
33 65
34 65
35 65
36 65
37 65
38 65
39 65
40 65
41 65
42 65
43 65
44 65
45 65
46 65
47 65
48 65
49 65
50 65
51 65
52 65
53 65
54 65
55 65
56 65
57 65
58 65
59 65
60 65
61 65
62 65
63 65
64 65
65 65
65
1 118
2 118
3 118
4 118
5 118
6 118
7 118
8 118
9 118
10 118
11 118
12 118
13 118
14 118
15 118
16 118
17 118
18 118
19 118
20 118
21 118
22 118
23 118
24 118
25 118
26 118
27 118
28 118
29 118
30 118
31 118
32 118
33 118
34 118
35 118
36 118
37 118
38 118
39 118
40 118
41 118
42 118
43 118
44 118
45 118
46 118
47 118
48 118
49 118
50 118
51 118
52 118
53 118
54 118
55 118
56 118
57 118
58 118
59 118
60 118
61 118
62 118
63 118
64 118
65 118
66 118
67 118
68 118
69 118
70 118
71 118
72 118
73 118
74 118
75 118
76 118
77 118
78

In [11]:


create_graphrels()#创建关系

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
recommand_eat 1 88
recommand_eat 2 88
recommand_eat 3 88
recommand_eat 4 88
recommand_eat 5 88
recommand_eat 6 88
recommand_eat 7 88
recommand_eat 8 88
recommand_eat 9 88
recommand_eat 10 88
recommand_eat 11 88
recommand_eat 12 88
recommand_eat 13 88
recommand_eat 14 88
recommand_eat 15 88
recommand_eat 16 88
recommand_eat 17 88
recommand_eat 18 88
recommand_eat 19 88
recommand_eat 20 88
recommand_eat 21 88
recommand_eat 22 88
recommand_eat 23 88
recommand_eat 24 88
recommand_eat 25 88
recommand_eat 26 88
recommand_eat 27 88
recommand_eat 28 88
recommand_eat 29 88
recommand_eat 30 88
recommand_eat 31 88
recommand_eat 32 88
recommand_eat 33 88
recommand_eat 34 88
recommand_eat 35 88
recommand_eat 36 88
recommand_eat 37 88
recommand_eat 38 88
recommand_eat 39 88
recommand_eat 40 88
recommand_eat 41 88
recommand_eat 42 88
recommand_eat 43 88
recommand_eat 44 88
recommand_eat 45 88
recommand_eat 46 88
recommand_eat 47 88
recommand_eat 48 88
recomma

In [16]:
export_data()#输出数据，可以选择不执行

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17


In [ ]:
# 成功完成知识图谱的构建